In [2]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [3]:
from lib.multitasking_lib import *
cache_type ='backtest_uni'

my_cache = cache_state('backtest_uni')

In [3]:
######################
# Initiate Backtest  #
######################


def store_algo(algo, algo_name="default"):
    redis_conn.hset("algos",algo_name, algo)
    algo_f = open("log/"+algo_name+".txt", "w")
    algo_f.write(algo)
    algo_f.close()
    
#my_cache.flushall()
def backtest_unitest(stocks, freq='1T',algo_name = 'haikin_1_new', fromDate='2019-4-8', toDate='2019-4-9', mode='full'):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq, 'mode':mode}

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)

try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler"], [kite_simulator, ohlc_tick_handler])
    
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


INFO:	Starting Handler: kite_simulator


True
['order_handlerlive', 'kite_ticker_handlerlive', 'kite_simulatorbacktest_web', 'ohlc_tick_handlerbacktest_web']
[<Process(Process-1, started)>]


INFO:	Starting Handler: ohlc_tick_handler
DEBUG:	kite_simulator: 1
ERROR:	kite_simulator: Invalid msg: 1
DEBUG:	ohlc_tick_handler(backtest_uni) - INIT: 1
TRADE: B: EN:  : 2019-04-08 09:19 : TCS -> 2058.0
TRADE: S: EX:  : 2019-04-08 09:23 : TCS -> 2060.85
TRADE: B: EN:  : 2019-04-08 09:28 : TCS -> 2056.4
TRADE: S: EX:  : 2019-04-08 09:41 : TCS -> 2046.35
TRADE: B: EN:  : 2019-04-08 09:51 : TCS -> 2039.35
TRADE: S: EX:  : 2019-04-08 10:07 : TCS -> 2044.0
TRADE: B: EN:  : 2019-04-08 10:10 : TCS -> 2047.0
TRADE: S: EX:  : 2019-04-08 10:13 : TCS -> 2045.85
TRADE: B: EN:  : 2019-04-08 10:18 : TCS -> 2045.1
TRADE: S: EX:  : 2019-04-08 10:20 : TCS -> 2045.55
TRADE: B: EN:  : 2019-04-08 10:23 : TCS -> 2044.15
TRADE: S: EX:  : 2019-04-08 10:26 : TCS -> 2043.95
TRADE: B: EN:  : 2019-04-08 10:28 : TCS -> 2042.95
TRADE: S: EX:  : 2019-04-08 10:29 : TCS -> 2042.75
TRADE: B: EN:  : 2019-04-08 10:33 : TCS -> 2044.7
TRADE: S: EX:  : 2019-04-08 10:36 : TCS -> 2045.6
TRADE: S: EN:  : 2019-04-08 10:39 : T

TRADE: B: EN:  : 2019-04-08 11:41 : TCS -> 2058.0
TRADE: S: EX:  : 2019-04-08 11:43 : TCS -> 2057.45
TRADE: B: EN:  : 2019-04-08 11:54 : TCS -> 2052.45
TRADE: S: EX:  : 2019-04-08 12:05 : TCS -> 2052.55
TRADE: B: EN:  : 2019-04-08 12:10 : TCS -> 2053.9
TRADE: S: EX:  : 2019-04-08 12:13 : TCS -> 2052.1
TRADE: B: EN:  : 2019-04-08 12:21 : TCS -> 2050.3
TRADE: S: EX:  : 2019-04-08 12:26 : TCS -> 2050.9
TRADE: B: EN:  : 2019-04-08 12:29 : TCS -> 2050.45
TRADE: S: EX:  : 2019-04-08 12:30 : TCS -> 2051.1
TRADE: S: EN:  : 2019-04-08 12:36 : TCS -> 2051.0
TRADE: B: EX:  : 2019-04-08 12:37 : TCS -> 2052.4
TRADE: S: EN:  : 2019-04-08 12:39 : TCS -> 2050.95
TRADE: B: EX:  : 2019-04-08 12:44 : TCS -> 2051.1
TRADE: S: EN:  : 2019-04-08 12:47 : TCS -> 2049.9
TRADE: B: EX:  : 2019-04-08 12:54 : TCS -> 2048.0
TRADE: S: EN:  : 2019-04-08 13:01 : TCS -> 2054.2
TRADE: B: EX:  : 2019-04-08 13:05 : TCS -> 2053.25
TRADE: B: EN:  : 2019-04-08 13:09 : TCS -> 2053.0
TRADE: S: EX:  : 2019-04-08 13:13 : TCS -> 2

In [ ]:
datetime.fromtimestamp(1589305326570/1000).strftime('%Y-%m-%d %H:%M:%S')

In [5]:
backtest_unitest(['TCS'])

In [7]:
my_cache.get('last_id_msg')

'1554708870000-0'

True

In [6]:
################################
###         Stop Trade       ###
################################

my_cache.delete('msgBufferQueue'+cache_postfix)
my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
time.sleep(1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
my_cache.set('last_id_msg',0)
jobs
jobs = []

False
['order_handlerlive', 'kite_ticker_handlerlive', 'kite_simulatorbacktest_web', 'ohlc_tick_handlerbacktest_web']


In [37]:
redis_conn.pubsub_channels()

['order_handlerlive',
 'kite_ticker_handlerlive',
 'kite_simulatorbacktest_web',
 'ohlc_tick_handlerbacktest_web']

In [ ]:
backtest_unitest(['TCS'])

In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [ ]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
chart_type ='haikin'

symbol='TCS'
my_cache = cache_state(cache_type)

#######  Hack for freq  ############
my_cache.setValue('TCS','freq','1T')
#####################################
dfohlc = my_cache.getOHLC(symbol)
print(dfohlc.tail())
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(dfohlc, trade_df, symbol, chart_type)

In [ ]:
cache_state('backtest_uni').getValue()

## Live Trade Debugging

In [185]:
live = threadManager('live', ["ohlc_tick_handler"], [ohlc_tick_handler])


INFO:	Starting Handler: ohlc_tick_handler
DEBUG:	ohlc_tick_handler(live) - INIT: 1
TRADE: S: EN:  : 2020-05-13 15:10 : WIPRO -> 189.3
TRADE: B: EX:  : 2020-05-13 15:16 : WIPRO -> 189.4
TRADE: S: EN:  : 2020-05-13 15:19 : WIPRO -> 190.0
TRADE: B: EX:  : 2020-05-13 15:20 : WIPRO -> 190.05
TRADE: S: EN:  : 2020-05-13 15:23 : WIPRO -> 189.95
TRADE: B: EX:  : 2020-05-13 15:24 : WIPRO -> 190.05
TRADE: S: EN:  : 2020-05-13 15:27 : WIPRO -> 189.9
TRADE: B: EX:  : 2020-05-13 16:18 : WIPRO -> 189.7
ERROR:	Could not read data from msgBufferQueue


In [116]:
live_cache.setValue('WIPRO','mode','paper')

0

In [184]:
live.job.terminate()
live_cache.set('WIPROliveTICK', pd.DataFrame().to_json(orient='columns'))
live_cache.setValue('WIPRO','last_processed',0)
live_cache.getValue('WIPRO')

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,WIPRO,1,1,1,189.7,0,0.20000000000001705,0.10531858873091997,-0.5000000000000568,-0.2635740643121017,...,0.0,0,189.7,0,SCANNING,paper,haikin_1_new,1T,minute,-1


In [73]:
#live_cache.getOHLC('WIPRO') 
print(live_cache.xrange('msgBufferQueuelive')[-2:])
print(datetime.fromtimestamp(int(live_cache.xrange('msgBufferQueuelive')[-1][0].split('-')[0] )/1000))


[('1589365238353-0', {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 189.7}]'}), ('1589365249733-0', {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 189.7}]'})]
2020-05-13 15:50:49.733000


In [175]:
live_cache.getOHLC('WIPRO').tail()

""


## Quick Backtest

In [ ]:
###########################################################
###                  Quick Backtest                     ###
###########################################################
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC, MIN, MAX, MINMAX

ohlc_get = lambda df, key: df.iloc[-1][key]
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)


################ Load Data ###################
from lib.charting_lib import *
cache_type = 'backtest_uni'
symbol='TCS'
my_cache = cache_state(cache_type)

###  Set Freq for Algo
my_cache.setValue('TCS','freq','1T')
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.head())

### Load Data
ohlc_data_temp = dfohlc
pd.set_option('display.max_rows', None)
################ Initialize Helper Functions ###################
trade_df1 = pd.DataFrame()
OPEN = ohlc_data_temp['open']
CLOSE = ohlc_data_temp['close']
HIGH = ohlc_data_temp['high']
LOW = ohlc_data_temp['low']
#VOLUME = ohlc_data_temp['volume']

(haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)

TIME = ohlc_data_temp.index[-1].minute+ohlc_data_temp.index[-1].hour*60

#REF = lambda df, i: df.iloc[-i-1]
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


def BUY(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["buy"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1

def SELL(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["sell"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1
    
################# Write Algo For Backtest Here ###################

#sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
#print(trade_df1)
sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
buy = (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))

logic = buy | sell

summary_df = trade_df1.copy(deep=True)
#summary_df['haopen'] = haOPEN[ logic ]
#summary_df['haclose'] = haCLOSE[ logic ]
summary_df['roc'] = ROC(CLOSE, 30)[logic]

#print(summary_df.sort_index())
#print(summary_df.sort_index().tail(10))
#print(ROC(CLOSE,3))
#################### Render Charts ###############################

trade_df1 = SELL(CLOSE, sell, trade_df1)
trade_df1 = BUY(CLOSE, buy, trade_df1)

#print(trade_df1.sort_index())
my_cache.setCache(symbol+cache_type+'Trade',trade_df1)
#my_cache.pushTrade(symbol, trade_df1)

chart_type ='haikin'
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(ohlc_data_temp, trade_df, symbol, chart_type)

In [ ]:
cache_type = 'backtest_web'
symbol='TCS'
my_cache = cache_state(cache_type)
my_cache.getTrades(symbol)

my_cache.get('TCSbacktest_webTrades')

In [ ]:
my_cache.getValue()

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

## Redis Console

In [4]:
from lib.multitasking_lib import *
#from lib.data_model_lib import *
import json
web_cache = cache_state('backtest_web')
live_cache = cache_state('live')
uni_cache = cache_state('backtest_uni')

pd.set_option('display.max_rows', None)

In [5]:
live_cache.pubsub_channels()

['order_handlerlive',
 'kite_ticker_handlerlive',
 'ohlc_tick_handlerlive',
 'kite_simulatorbacktest_web',
 'ohlc_tick_handlerbacktest_web']

In [ ]:
df = live_cache.getValue()

In [ ]:
live_cache.getOHLC('TCS')

In [ ]:
trades_df = live_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

### GetData

In [ ]:
logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [ ]:
ohlc_data[stock].head()

# Kite Connect

In [8]:
from kiteconnect import KiteConnect
kite_api_key = 'b2w0sfnr1zr92nxm'
live_cache.set('KiteAPIKey',kite_api_key)
access_token = live_cache.get('access_token')
print(access_token)
print(live_cache.get('KiteAPIKey'))

kite = KiteConnect(api_key=kite_api_key)
kite.set_access_token(access_token)

KRtj5s4LxJ6Ihv7UvH21jraPo84G19ij
b2w0sfnr1zr92nxm


In [9]:
kite.access_token

'KRtj5s4LxJ6Ihv7UvH21jraPo84G19ij'

In [ ]:
kite.profile()

In [ ]:
cancel_order('CIPLA')

In [ ]:
orders_df = pd.DataFrame(kite.orders())
orders_df[['order_id','status','tradingsymbol','transaction_type','quantity','tag','price']]

In [ ]:
symbol = 'INFY'
live_cache.add(symbol)
token = int(live_cache.hmget('eq_token',symbol)[0])
live_cache.publish('live_trade_handlerlive', json.dumps({'cmd':'add','value':[token],'mode':'ltp'}))

live_cache.setValue('INFY','qty','1')
live_cache.setValue('INFY','SL %','0.4')
live_cache.setValue('INFY','TP %','1')
live_cache.setValue('INFY','algo','haikin_1_new')
live_cache.setValue('INFY','freq','1T')

live_cache.setValue('INFY','mode','live')

In [10]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,WIPRO,1,1,1,0,0,0,0.0,0,0.0,...,0.0,0.0,0.0,1999-01-01,INIT,PAUSE,haikin_1_new,1T,minute,200513002688712


In [ ]:
live_cache.setValue('INFY','mode','paper')
live_cache.setValue('INFY','qty','1')
live_cache.setValue('INFY','SL %','0.4')
live_cache.setValue('INFY','TP %','1')
live_cache.setValue('INFY','algo','haikin_1_new')
live_cache.setValue('INFY','freq','1T')

In [ ]:
live_cache.setValue('INFY','qty','1')
live_cache.setValue('INFY','SL %','0.4')
live_cache.setValue('INFY','TP %','1')
live_cache.setValue('INFY','algo','haikin_1_new')
live_cache.setValue('INFY','freq','1T')

In [ ]:
live_cache.setValue('CIPLA','mode','PAUSE')

In [ ]:
position_df = pd.DataFrame(kite.positions()['net'])
position_df

In [ ]:
pd.DataFrame(kite.positions()['day'])

In [ ]:
orders_df = pd.DataFrame()
for msg in live_cache.xrange('notificationQueuelivenew'):
    tmp_df = pd.DataFrame.from_dict(json.loads(msg[1]['data']), orient='index').transpose()
    orders_df = orders_df.append(tmp_df)
    #print(msg)
orders_df

In [ ]:
kite.margins()

### Kite Ticker Handler Manipulation

In [ ]:
#live_cache.publish('live_trade_handlerlive','profile')
live_cache.publish('live_trade_handlerlive','status')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','INIT')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','CLOSE')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','START')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','profile')

In [ ]:
live_cache.smembers('live_token')

In [ ]:
values = list(map(int,live_cache.smembers('ticker_list')))
values

In [ ]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'add','value':values,'mode':'ltp'}))

In [ ]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'remove','value':[1270529, 969473, 1510401],'mode':'quote'}))


In [ ]:
pd.DataFrame(kite.orders())

### OrderHandler Monitor and Manipulation

In [96]:
# Pause Ordering
live_cache.publish('order_handlerlive', 'pause')

1

In [182]:
# Resume Ordering
live_cache.publish('order_handlerlive', 'resume')

1

In [97]:
# Buy
live_cache.publish('order_handlerlive',json.dumps({'cmd':'buy','symbol':'WIPRO','price':180,'qty':1}))

1

In [12]:
# Sell
live_cache.publish('order_handlerlive',json.dumps({'cmd':'sell','symbol':'WIPRO','price':190,'qty':1}))

1

In [ ]:
# Cancel
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancel','symbol':'WIPRO'}))

In [ ]:
live_cache.publish('order_handlerlive',json.dumps({'cmd':'getOrders'}))

In [33]:
## Cancel All the orders
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancelAll'}))

1

In [ ]:
#from lib.kite_wrapper_lib import *
cancel_order(kite,['CIPLA'])

In [ ]:
buy_limit('WIPRO',188.5,1)

In [ ]:
sell_limit('WIPRO',188.7,1)

In [11]:
live_cache.getOHLC('WIPRO').tail(10)

""


In [ ]:
kite.cancel_order(kite.VARIETY_REGULAR, 200511002924441, parent_order_id=None)

In [ ]:
kite.convert_position(kite.EXCHANGE_NSE, 'WIPRO', transaction_type, position_type, quantity, old_product, new_product)


In [ ]:
kite.modify_order(kite.VARIETY_REGULAR, 200511003019322, parent_order_id=None, quantity=None, price=187.4, order_type=None, trigger_price=None, validity=None, disclosed_quantity=None)


In [ ]:
msg = {'data': '[{"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 184.7, "last_quantity": 5, "average_price": 187.07, "volume": 4393242, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 185.0, "high": 189.95, "low": 184.1, "close": 184.0}, "change": 0.38043478260868946}, {"tradable": true, "mode": "quote", "instrument_token": 2953217, "last_price": 1935.3, "last_quantity": 25, "average_price": 1939.47, "volume": 2449395, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 1909.0, "high": 1951.4, "low": 1909.0, "close": 1893.4}, "change": 2.2129502482306886}]'}

In [ ]:
msg['data']

In [ ]:
val = json.loads(msg['data'])

##### Reset state for tradejob

In [ ]:
live_cache.setValue('WIPRO','state','SCANNING')

### Monitor: OHLC Tick Handler

In [ ]:
live_cache.get('last_id_msg')

### Live Cache State Monitor and manipulation

In [ ]:
live_cache.getValue()

In [ ]:
stock_key = 'WIPRO'
live_cache.add(stock_key, True) #TODO: While adding stock to table use True
live_cache.setValue(stock_key, 'algo', 'haikin_1_new')
live_cache.setValue(stock_key, 'freq', '1T')
live_cache.setValue(stock_key, 'qty', 1)
live_cache.setValue(stock_key, 'SL %', 1)
live_cache.setValue(stock_key, 'TP %', 1)
live_cache.setValue(stock_key, 'P&L', 0)
live_cache.setValue(stock_key, 'Total P&L', 0)
live_cache.setValue(stock_key, 'price', 0)
live_cache.setValue(stock_key, 'hdf_freq', 'minute')
live_cache.getValue()

In [ ]:
live_cache.remove('AXISBANK')
live_cache.getValue()

#### Get token for Stock symbol

In [ ]:
live_cache.hmget('eq_token',['CIPLA','INFY'])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO','AXISBANK'])

#### Peek into OHLC buffer

In [ ]:
live_cache.getOHLC('WIPRO') 

#### msgBufferQueue Monitoring

In [10]:
live_cache.xrange('msgBufferQueuelive')[-1]

('1589370308339-0',
 {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 189.7}]'})

In [13]:
datetime.fromtimestamp(int(live_cache.xrange('msgBufferQueuelive')[-1][0].split('-')[0] )/1000)

datetime.datetime(2020, 5, 13, 17, 15, 8, 339000)

#### TickerList Manipulaitons

In [ ]:
live_cache.sadd('ticker_list','2953217')
live_cache.sadd('ticker_list','969473')

live_cache.sadd('ticker_list','1510401')

In [6]:
values = list(map(int,live_cache.smembers('ticker_list')))
print(values)
live_cache.hmget('eq_token', values)

[969473]


['WIPRO']

In [ ]:
live_cache.srem('ticker_list',1510401)

In [ ]:
live_cache.sismember('ticker_list',4268801)

In [ ]:
live_cache.delete('ticker_list')

In [ ]:
live_cache.getValue()

In [ ]:
live_cache.get('Kite_Status')

### Update Redis with Instrument list

##### Download fresh data and store in cache

In [ ]:
instruments_df = pd.DataFrame(kite.instruments())

In [ ]:
instruments_df.to_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df.columns

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),'instrument_token'].values

##### Use cache

In [ ]:
instruments_df = pd.read_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),:].values

eq_nse.set_index('instrument_token')

In [ ]:
live_cache.delete('eq_token')

In [ ]:
eq_nse.head().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('tradingsymbol').transpose().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('instrument_token').transpose().to_dict(orient='records')[0]

##### Update redis with tokens

In [ ]:
live_cache.hmset('eq_token',eq_nse.set_index('instrument_token').transpose().to_dict(orient='records')[0])
live_cache.hmset('eq_token',eq_nse.set_index('tradingsymbol').transpose().to_dict(orient='records')[0])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO'])

In [ ]:
live_cache.hgetall('eq_token')

In [ ]:
live_cache.hmget('eq_token',[2953217, 969473])